In [4]:
import CONST
import wandb
from pprint import pprint

from entities import WoundDataset
from models.trainer.diffuser import Diffuser

In [13]:
# Login wandb
wandb_key = "a8b5a7676a58d9b5b1e686fd9d349bc25f18d07c"
wand_logged = wandb.login(key=wandb_key)

wandb: Currently logged in as: qhuy0168. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/nathanbui/.netrc


In [14]:
# Define sweep configuration
sweep_config = {
    "project": "DDPM_hyper_tuning",
    "method": "bayes",
    "metric": {
        "name": "train_mse_loss",
        "goal": "minimize",
    },
    "parameters": {
        "learning_rate": {"distribution": "uniform", "min": 5e-5, "max": 3e-4},
        "noise_steps": {"values": [50, 100, 500, 1000]},
        "variance_schedule_type": {"values": ["linear", "quadratic", "sigmoid", "cosine"]},
        "attn_heads": {"values": [1, 2, 4]}
    }
}

pprint(sweep_config)

{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'train_mse_loss'},
 'parameters': {'attn_heads': {'values': [1, 2, 4]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.0003,
                                  'min': 5e-05},
                'noise_steps': {'values': [50, 100, 500, 1000]},
                'variance_schedule_type': {'values': ['linear',
                                                      'quadratic',
                                                      'sigmoid',
                                                      'cosine']}},
 'project': 'DDPM_hyper_tuning'}


In [17]:
# Initialise sweep controller on WandB server
# After initialised, we do not need to run this again
sweep_id = wandb.sweep(sweep_config, project="DDPM_hyper_tuning")

Create sweep with ID: 7saxy261
Sweep URL: https://wandb.ai/qhuy0168/DDPM_hyper_tuning/sweeps/7saxy261


In [18]:
print(sweep_id)

7saxy261


In [8]:
# Define the agent
# Run this on agent computers
def train():
    """
    Training trigger for hyper-parameter optimisation
    :return: None
    """
    # Init the run
    run = wandb.init(project="DDPM_hyper_tuning")

    # Get the configuration for instance
    config = wandb.config

    # Similar approach to full train
    # The data
    dataset = WoundDataset(
        image_dir=CONST.PROCESSED_IMAGES_DIR,
        segment_dir=CONST.PROCESSED_SEGMENT_DIR,
        target_tensor_size=CONST.DIFFUSER_SETTINGS.INPUT_SIZE,
        embedding_dir=CONST.PROCESSED_EMBEDDING_DIR
    )

    diffuser = Diffuser(
        dataset=dataset,
        batch_size=8,
        num_workers=2,
        epochs=500,
        run_name=CONST.DIFFUSER_SETTINGS.RUN_NAME,
        output_dir=CONST.DIFFUSER_SETTINGS.OUTPUT_DIR,
        max_lr=config.learning_rate,
        noise_steps=config.noise_steps,
        variance_schedule_type=config.variance_schedule_type,
        attn_heads=config.attn_heads,
        wandb_run=run,
    )

    diffuser.fit()

In [ ]:
# Start the sweep agent
wandb.agent(sweep_id="7saxy261", project="DDPM_hyper_tuning", function=train, count=10)